[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/instant-ngp-colab/blob/main/ceres_solver_build.ipynb)

In [ ]:
!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev
    
!wget http://ceres-solver.org/ceres-solver-2.1.0.tar.gz

In [ ]:
%cd /content
!tar zxf ceres-solver-2.1.0.tar.gz
!mkdir ceres-bin
%cd ceres-bin
!cmake /content/ceres-solver-2.1.0 -DEXPORT_BUILD_DIR=ON -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF
!make
!make install

In [ ]:
%cd /content
!git clone https://github.com/colmap/colmap.git
%cd colmap
!git checkout dev
!mkdir build
%cd build
!cmake ..
!sudo make install

In [ ]:
!pip install huggingface-hub
from huggingface_hub import create_repo, upload_folder
hf_token = ""
repo_id = "camenduru/ceres-solver"
create_repo(repo_id, private=True, token=hf_token)
upload_folder(folder_path="/content/build", path_in_repo="", repo_id=repo_id, commit_message=f"ceres-solver and colmap", token=hf_token)

In [ ]:
%cd /content
!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/instant-ngp.zip
!unzip /content/instant-ngp.zip -d instant-ngp

!pip install commentjson
%cd /content/instant-ngp
import os

In [ ]:
!python ./scripts/colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 16 --images /content/drive/MyDrive/bird

In [ ]:
scene_path = "/content/instant-ngp/data/nerf/bird" #@param {type:"string"}
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)

train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(scene_path, f"{train_steps}.msgpack")
!python ./scripts/run.py --scene {scene_path} --mode nerf --n_steps {train_steps} --save_snapshot {snapshot_path}